전체 디스크립터 중 모든 값이 0인 디스크립터의 비율 확인

In [1]:
import os
import sys
import argparse
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import open3d.visualization as vis
import torch
from scipy.spatial.distance import cdist

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
opt = argparse.Namespace(
    fpfh_normal_radiuse = 0.5,
    fpfh_descriptors_radiuse = 1.25,
    seq_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    mdgat_path = './KITTI',
    kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
    save_path = './KITTI/keypoints/tsf_256_FPFH_nr05_dr125',
    memory_is_enough = True,
    mutual_check = True,
    transform_opt = 0
)

In [16]:
save_path = './KITTI/keypoints/tsf_256_FPFH_nr05_dr125'
seq_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
idx_list = range(0, 10000)

entire_norm_num_list = []
zeros_norm_num_list = []
max_zeros_num_list = []

for seq in seq_list:
    seq_norm_num = 0
    seq_zeros_num = 0
    max_zeros_num = 0
    for idx in idx_list:
        try:
            file = os.path.join(save_path, '%02d' % seq, '%06d.bin' % idx)
            pc = np.fromfile(file, dtype=np.float64)
            desc = pc.reshape(-1, 37)[:, 3:]
            norm = np.linalg.norm(desc, axis=1)
            seq_norm_num += len(norm)
            seq_zeros_num += len(norm) - np.count_nonzero(norm)
            if max_zeros_num < len(norm) - np.count_nonzero(norm):
                max_zeros_num = len(norm) - np.count_nonzero(norm)
        except:
            # print("end of sequence %02d" % seq)
            entire_norm_num_list.append(seq_norm_num)
            zeros_norm_num_list.append(seq_zeros_num)
            max_zeros_num_list.append([max_zeros_num, 256])
            break

for seq in seq_list:
    # print((entire_norm_num_list[seq] / zeros_norm_num_list[seq]))
    print("sequence %02d" % seq, 
          ", entire norm num: %d" % entire_norm_num_list[seq], 
          ", zeros norm num: %d" % zeros_norm_num_list[seq], 
          ", ratio of zeros: %.4f" % (entire_norm_num_list[seq] / zeros_norm_num_list[seq]), 
          ", max zeros num: %d / %d" % (max_zeros_num_list[seq][0], max_zeros_num_list[seq][1]))


sequence 00 , entire norm num: 581248 , zeros norm num: 78248 , ratio of zeros: 7.4283 , max zeros num: 75 / 256
sequence 01 , entire norm num: 140928 , zeros norm num: 46401 , ratio of zeros: 3.0372 , max zeros num: 81 / 256
sequence 02 , entire norm num: 596608 , zeros norm num: 83209 , ratio of zeros: 7.1700 , max zeros num: 69 / 256
sequence 03 , entire norm num: 102528 , zeros norm num: 28365 , ratio of zeros: 3.6146 , max zeros num: 82 / 256
sequence 04 , entire norm num: 34688 , zeros norm num: 8941 , ratio of zeros: 3.8797 , max zeros num: 81 / 256
sequence 05 , entire norm num: 353408 , zeros norm num: 47335 , ratio of zeros: 7.4661 , max zeros num: 62 / 256
sequence 06 , entire norm num: 140928 , zeros norm num: 54358 , ratio of zeros: 2.5926 , max zeros num: 83 / 256
sequence 07 , entire norm num: 140928 , zeros norm num: 18614 , ratio of zeros: 7.5711 , max zeros num: 53 / 256
sequence 08 , entire norm num: 521088 , zeros norm num: 92192 , ratio of zeros: 5.6522 , max zeros

양쪽 데이터 뜯어보기

In [21]:
save_path = './KITTI/keypoints/tsf_256_FPFH_nr05_dr125'
seq_list = [0]
# idx_list = range(0, 1)
idx_list = [10]

for seq in seq_list:
    seq_norm_num = 0
    seq_zeros_num = 0
    max_zeros_num = 0
    for idx in idx_list:
        file = os.path.join(save_path, '%02d' % seq, '%06d.bin' % idx)
        pc = np.fromfile(file, dtype=np.float64)
        desc = pc.reshape(-1, 37)[:, 3:]
        norm = np.linalg.norm(desc, axis=1)
        np.set_printoptions(threshold=np.inf)
        print(desc, norm)

[[1.60922691e-003 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [9.37338564e-004 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000

이거 뭔가 이상하다. 이게 말이 되는 디스크립터인가?